## Dataset Download and Extraction

In this section, we are downloading the chest X-ray dataset for pneumonia detection from Kaggle and extract it for further processing. We'll use the Kaggle API to facilitate the download and the Python `ZipFile` library for extraction.

### Kaggle Configuration

Before proceeding, we have to make sure you have set up your Kaggle API credentials. You can configure your API credentials by setting the `KAGGLE_CONFIG_DIR` environment variable to the directory containing your `kaggle.json` file.

In [2]:
from zipfile import ZipFile

In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [4]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

100% 2.29G/2.29G [01:51<00:00, 20.2MB/s]
100% 2.29G/2.29G [01:51<00:00, 22.0MB/s]


Once the dataset is downloaded, we'll extract its contents into a directory named "chest_xray."

In [5]:
with ZipFile('/content/chest-xray-pneumonia.zip','r') as zipobj:
  zipobj.extractall("/content/chest_xray")

In [6]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


### Importing Libraries

We start by importing the essential libraries required for this project. These libraries include TensorFlow, Keras, NumPy, Pandas, and additional utilities for image processing.


In [7]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.utils import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
from keras.layers import LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, Add, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input, Lambda
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

### Loading the Dataset

We start by loading the dataset from the specified directory using `image_dataset_from_directory`. This function allows us to load images directly from directories and infer labels based on subdirectories. We are loading training, testing and validation datasets.

### Data Normalization
To prepare the data for training, we normalize the pixel values of the images. Normalization typically involves scaling pixel values to a range between 0 and 1. This helps in better convergence during model training. We have applied the same to the test and validation dataset as well

In [8]:
train_dataset = image_dataset_from_directory("/content/chest_xray/chest_xray/train",
                             labels="inferred",
                             label_mode="categorical",
                             class_names=["NORMAL","PNEUMONIA"],
                             batch_size=32,
                             image_size=(512,512),
                             color_mode="grayscale",
                             seed=42)
train_dataset = train_dataset.map(lambda x, y: (x / 255.0, y))

Found 5216 files belonging to 2 classes.


In [9]:
validation_dataset = image_dataset_from_directory("/content/chest_xray/chest_xray/val",
                             labels="inferred",
                             label_mode="categorical",
                             class_names=["NORMAL","PNEUMONIA"],
                            #  batch_size=32,
                             image_size=(512,512),
                             color_mode="grayscale",
                             seed=42)
validation_dataset = validation_dataset.map(lambda x, y: (x / 255.0, y))

Found 16 files belonging to 2 classes.


In [10]:
test_dataset = image_dataset_from_directory("/content/chest_xray/chest_xray/test",
                             labels="inferred",
                             label_mode="categorical",
                             class_names=["NORMAL","PNEUMONIA"],
                            #  batch_size=32,
                             image_size=(512, 512),
                             color_mode="grayscale",
                             seed=42)
test_dataset = test_dataset.map(lambda x, y: (x / 255.0, y))

Found 624 files belonging to 2 classes.


## Building and Training the Classification Model

In this section, we define the architecture of our chest X-ray classification model and train it using the preprocessed training dataset. Let's break down the steps involved in constructing and training the model:

### Model Architecture

We construct a convolutional neural network (CNN) using Keras to learn features from chest X-ray images. Below is a detailed explanation of the model layers:
The model consists of convolutional layers with ReLU activation functions and

- LeakyReLU layers to introduce non-linearity.

- Max-pooling layers downsample the feature maps.

- Dropout layers are added to prevent overfitting.

### Model Compilation
We compile the model by specifying the optimizer, loss function, and evaluation metric:

- We use the Adam optimizer.

- Binary cross-entropy loss is chosen for binary classification.

- Accuracy is the evaluation metric.

### Training the Model
The model is trained using the training dataset, with validation data provided for monitoring training progress:

We train the model for 100 epochs.

In [ ]:
model = keras.models.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3,3), activation="relu", input_shape = (256, 256, 1)))
model.add(LeakyReLU(alpha=0.2))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(64, kernel_size=(3,3), activation="relu"))
model.add(LeakyReLU(alpha=0.2))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(128, kernel_size=(3,3), activation="relu"))
model.add(LeakyReLU(alpha=0.2))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(256, kernel_size=(3,3), activation="relu"))
model.add(LeakyReLU(alpha=0.2))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(128, kernel_size=(3,3), activation="relu"))
model.add(LeakyReLU(alpha=0.2))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(64, kernel_size=(3,3), activation="relu"))
model.add(LeakyReLU(alpha=0.2))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Dropout(0.5))
# model.add(layers.Conv2D(32, kernel_size=(3,3), activation="relu"))
# model.add(LeakyReLU(alpha=0.2))
# model.add(layers.MaxPooling2D(2,2))
# model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation="relu"))
model.add(layers.Dense(2, activation="sigmoid"))

model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

# model.summary()
history = model.fit(train_dataset, epochs=100, validation_data=validation_dataset)

In [ ]:
model.save("content/chest_xray/model2.keras")

## Model Evaluation on Test Data

In this section, we evaluate the performance of our chest X-ray classification model on a separate test dataset. The model has been trained on the training dataset, and now we assess how well it generalizes to unseen data.

### Model Evaluation Code

We use the `evaluate` method to assess the model's performance on the test dataset:

In [ ]:
model.evaluate(test_dataset)

20/20 [==============================] - 2s 74ms/step - loss: 0.7662 - accuracy: 0.8029


[0.7662030458450317, 0.8028846383094788]

### Evaluation Output
Upon running the evaluation code, we obtain the following output:

The evaluation was performed on 20 batches of test data.
loss: `0.7662` represents the computed loss value.
accuracy: `0.8029` indicates the accuracy of the model on the test dataset.

An accuracy of approximately 80.29% suggests that the model is capable of distinguishing between normal and pneumonia cases with a good level of accuracy.

## Residual Block for CNN

In this section, we define a custom residual block function for our CNN arhitecture to deal with the vanishing gradient problem.

We have defined a custom residual block function, `residual_block`, which takes the following parameters:

- `x`: Input tensor.
- `filters`: The number of filters in the convolutional layers.
- `kernel_size`: The size of the convolutional kernel.
- `stride`: The stride for the convolutional layers (default is 1).


we are using the `residual_block` function to create a residual block within our CNN architecture. These blocks help in the training of deep networks and enable the construction of deep CNNs with improved gradient flow.

In [ ]:
def residual_block(x, filters, kernel_size, stride=1):
    identity = x

    x = Conv2D(filters, kernel_size, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)

    if stride != 1 or identity.shape[-1] != filters:
        identity = Conv2D(filters, (1, 1), strides=stride, padding='same')(identity)

    x = Add()([x, identity])
    x = Activation('relu')(x)
    return x

## Building a Residual Network (ResNet) Model

We define a convolutional neural network (CNN) model using the ResNet architecture. This helps us to train very deep networks effectively by employing residual blocks, which help address the vanishing gradient problem.

### Model Architecture

We define a function `build_resnet` that constructs the ResNet model. Here's an overview of the model architecture:

1. **Input Layer**:
   - The model starts with an input layer that takes images of shape `(256, 256, 1)`.

2. **Initial Convolution and Pooling**:
   - A 2D convolutional layer with 64 filters, a `(7, 7)` kernel size, and `strides=(2, 2)` is applied.
   - Batch normalization and ReLU activation follow the convolution.
   - A max-pooling layer with `(3, 3)` pooling size and `strides=(2, 2)` is used for downsampling.

3. **Residual Blocks**:
   - We employ a series of residual blocks using the `residual_block` function:
     - Two residual blocks with 64 filters and `(3, 3)` kernel size.
     - Two residual blocks with 128 filters and `(3, 3)` kernel size, with a stride of 2 for downsampling.
     - Two residual blocks with 256 filters and `(3, 3)` kernel size, with a stride of 2 for downsampling.
     - Two residual blocks with 512 filters and `(3, 3)` kernel size, with a stride of 2 for downsampling.

4. **Global Average Pooling and Output Layer**:
   - Global average pooling is applied to reduce the spatial dimensions.
   - A fully connected layer with softmax activation produces the final output.

### Model Compilation

We compile the ResNet model with the following settings:
- We use the Adam optimizer.
- Categorical cross-entropy loss is chosen for multi-class classification.
- Accuracy is used as the evaluation metric.

### Model Summary
To get an overview of the model's architecture and the number of trainable parameters, we print the model summary using model.summary().

In [ ]:
def build_resnet(input_shape, num_classes):
    input_tensor = Input(shape=input_shape)

    x = Conv2D(64, (7, 7), padding='same', strides=(2, 2))(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = residual_block(x, 64, (3, 3))
    x = residual_block(x, 64, (3, 3))
    x = residual_block(x, 128, (3, 3), stride=2)
    x = residual_block(x, 128, (3, 3))
    x = residual_block(x, 256, (3, 3), stride=2)
    x = residual_block(x, 256, (3, 3))
    x = residual_block(x, 512, (3, 3), stride=2)
    x = residual_block(x, 512, (3, 3))

    x = Flatten()(x)
    x = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=input_tensor, outputs=x)
    return model

input_shape = (256, 256, 1)
num_classes = 2
resnet_model = build_resnet(input_shape, num_classes)

resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

resnet_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 64  3200        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 128, 128, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                             

### Model Training

We train the ResNet model on the training dataset using the `fit` method. The training process includes 100 epochs to iteratively learn from the data:

In [ ]:
history = resnet_model.fit(train_dataset, epochs=100, validation_data=validation_dataset)
resnet_model.save("/content/drive/MyDrive/ChestXraay/resne_model.keras")

Epoch 1/100
163/163 [==============================] - 63s 217ms/step - loss: 0.8969 - accuracy: 0.9053 - val_loss: 3.3564 - val_accuracy: 0.5000
Epoch 2/100
163/163 [==============================] - 35s 204ms/step - loss: 0.1981 - accuracy: 0.9442 - val_loss: 9.6286 - val_accuracy: 0.5000
Epoch 3/100
163/163 [==============================] - 37s 213ms/step - loss: 0.0933 - accuracy: 0.9701 - val_loss: 13.8233 - val_accuracy: 0.5000
Epoch 4/100
163/163 [==============================] - 37s 221ms/step - loss: 0.0735 - accuracy: 0.9749 - val_loss: 0.5602 - val_accuracy: 0.8750
Epoch 5/100
163/163 [==============================] - 36s 206ms/step - loss: 0.0714 - accuracy: 0.9755 - val_loss: 18.6334 - val_accuracy: 0.5000
Epoch 6/100
163/163 [==============================] - 36s 215ms/step - loss: 0.3145 - accuracy: 0.9381 - val_loss: 128.4397 - val_accuracy: 0.5000
Epoch 7/100
163/163 [==============================] - 36s 209ms/step - loss: 0.0848 - accuracy: 0.9712 - val_loss: 0.12

### Model Evaluation Code

We use the `evaluate` method to assess the model's performance on the test dataset:

In [ ]:
resnet_model.evaluate(test_dataset)

20/20 [==============================] - 3s 88ms/step - loss: 5.6391 - accuracy: 0.7516


[5.639106750488281, 0.7516025900840759]

### Evaluation Output
Upon running the evaluation code, we obtain the following output:

20/20 indicates that the evaluation was performed on 20 batches of test data.
loss: `5.6391` represents the computed loss value.
accuracy: `0.7516` indicates the accuracy of the model on the test dataset.

The accuracy has been decreased from our previous model.

## Custom Image Classification Model with VGG16 Base

We are creating a custom image classification model using the VGG16 architecture as a base model.

This custom VGG16-based model differs from the previously built ResNet-based model in terms of architecture. VGG16 is known for its simplicity with a fixed architecture, while ResNet introduces residual blocks to facilitate training of very deep networks.

The choice between these models often depends on the specific dataset and problem you are solving. VGG16 can be more suitable for smaller datasets and tasks with moderate complexity, whereas ResNet may excel in very deep networks and complex problems.

### Model Architecture

1. **Base Model**: We load a pre-trained VGG16 model (excluding the top fully connected layers) using the `VGG16` function with `include_top=False` and `weights='imagenet'`. The input shape is set to `(512, 512, 3)`.

2. **Input Layer Modification**: As the VGG16 model expects RGB images with three channels, we create a custom input layer for grayscale images with a single channel. We use a custom layer, `repeat_channels`, to duplicate the single channel into three channels to match the VGG16 input shape.

3. **Pass Through Pre-trained Model**: The modified input is passed through the VGG16 base model. The pre-trained layers of VGG16 are frozen, ensuring that their weights remain unchanged during training.

4. **Custom Classifier Head**: On top of the base model, we add custom layers for classification:
   - Global Average Pooling to reduce the spatial dimensions.
   - Multiple dense layers with ReLU activation and dropout for feature extraction and regularization.
   - The final output layer with two units for multiclass classification using softmax activation.

### Model Compilation

We compile the model using the Adam optimizer and categorical cross-entropy loss, suitable for multiclass classification:

In [ ]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(512, 512, 3))

def repeat_channels(x):
    return tf.concat([x, x, x], axis=-1)

input_layer = Input(shape=(512, 512, 1))

rgb_input = Lambda(repeat_channels)(input_layer)

x = base_model(rgb_input)

for layer in base_model.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 512, 512, 1)]     0         
                                                                 
 lambda_2 (Lambda)           (None, 512, 512, 3)       0         
                                                                 
 vgg16 (Functional)          (None, 16, 16, 512)       14714688  
                                                                 
 global_average_pooling2d_2   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_16 (Dense)            (None, 256)               131328    
                                                                 
 dropout_13 (Dropout)        (None, 256)               0         
                                                           

## Early Stopping and Model Saving

In this section, we introduce early stopping as a technique during model training to prevent overfitting and save the trained model after training. Let's break down the key components of this code:

### Early Stopping

We define an `EarlyStopping` callback, which monitors the validation loss (`val_loss`). If the validation loss does not improve for a specified number of consecutive epochs (`patience`), training will be stopped. The `restore_best_weights` parameter ensures that the model's best weights are restored.

### Model Training
We use a try-except-finally block to handle model training with early stopping. The model is trained with the fit method, specifying the training dataset, number of epochs, batch size, and validation dataset.

We train the model for a maximum of 50 epochs but stop early if the validation loss does not improve for 10 consecutive epochs, as specified by the EarlyStopping callback.
The try block captures any potential early stopping triggered by the callback or a manual interruption.
In the finally block, we save the trained model to a specified path for later use.

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    restore_best_weights=True
)
#model.fit(train_dataset, epochs=25, validation_data=validation_dataset)
#model.save("/content/drive/MyDrive/ChestXraay/VGG16_model2.keras")

try:
  history = model.fit(train_dataset,
                      epochs=50, batch_size=32,
                      validation_data = validation_dataset,
                      callbacks=[early_stopping])
except KeyboardInterrupt:
  print("\n\nTraining Stopped")
finally:
  model.save("/content/drive/MyDrive/ChestXraay/VGG16_model2.keras")

Epoch 1/50
163/163 [==============================] - 110s 644ms/step - loss: 0.4386 - accuracy: 0.7937 - val_loss: 0.8380 - val_accuracy: 0.7500
Epoch 2/50
163/163 [==============================] - 106s 640ms/step - loss: 0.2775 - accuracy: 0.8825 - val_loss: 0.9156 - val_accuracy: 0.6250
Epoch 3/50
163/163 [==============================] - 106s 641ms/step - loss: 0.2606 - accuracy: 0.8940 - val_loss: 0.6974 - val_accuracy: 0.7500
Epoch 4/50
163/163 [==============================] - 108s 640ms/step - loss: 0.2438 - accuracy: 0.9032 - val_loss: 0.6966 - val_accuracy: 0.6250
Epoch 5/50
163/163 [==============================] - 106s 640ms/step - loss: 0.2336 - accuracy: 0.9091 - val_loss: 0.5707 - val_accuracy: 0.7500
Epoch 6/50
163/163 [==============================] - 106s 639ms/step - loss: 0.2319 - accuracy: 0.9145 - val_loss: 0.5226 - val_accuracy: 0.6875
Epoch 7/50
163/163 [==============================] - 107s 639ms/step - loss: 0.2119 - accuracy: 0.9224 - val_loss: 0.6926 -

### Training Output
  - Training continues to epoch 16.
  - In this epoch, the training accuracy is approximately 92.87%, and the training loss is 0.1984.
  - The validation accuracy remains at 62.50%, and the validation loss is 0.7139.

  Training stopped at 16 epochs as the validation loss remains unchanged after 10 iterations.

In [ ]:
model = keras.saving.load_model("/content/drive/MyDrive/ChestXraay/VGG16_model2.keras")

In [ ]:
model.evaluate(train_dataset)

163/163 [==============================] - 132s 639ms/step - loss: 0.2308 - accuracy: 0.9484


[0.2308228611946106, 0.9484279155731201]

In [ ]:
model.evaluate(test_dataset)

20/20 [==============================] - 25s 1s/step - loss: 0.3642 - accuracy: 0.8125


[0.36424189805984497, 0.8125]

### Model Evaluation Code

We use the `evaluate` method to assess the model's performance on the test dataset.
- 20/20 indicates that the evaluation was performed on 20 batches of test data.
- loss: 0.3642 represents the computed loss value.
- accuracy: 0.8125 indicates the accuracy of the model on the test dataset.

We can see that the accuracy is acheived around one percent extra from our first model which isn't a lot, but when we compare that we have achieved this by using just 16 epochs instead of 100 this does work well compared to our first model.

Let's see what happens if change the early stopping by looking at `loss` instead of `val_loss`.

In [11]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(512, 512, 3))

def repeat_channels(x):
    return tf.concat([x, x, x], axis=-1)

input_layer = Input(shape=(512, 512, 1))

rgb_input = Lambda(repeat_channels)(input_layer)

x = base_model(rgb_input)

for layer in base_model.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

58889256/58889256 [==============================] - 4s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 512, 512, 1)]     0         
                                                                 
 lambda (Lambda)             (None, 512, 512, 3)       0         
                                                                 
 vgg16 (Functional)          (None, 16, 16, 512)       14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0      

In [12]:
early_stopping = EarlyStopping(
    monitor='loss',
    patience=10,
    verbose=1,
    restore_best_weights=True
)
#model.fit(train_dataset, epochs=25, validation_data=validation_dataset)
#model.save("/content/drive/MyDrive/ChestXraay/VGG16_model2.keras")

try:
  history = model.fit(train_dataset,
                      epochs=50, batch_size=32,
                      validation_data = validation_dataset,
                      callbacks=[early_stopping])
except KeyboardInterrupt:
  print("\n\nTraining Stopped")
finally:
  model.save("/content/drive/MyDrive/ChestXraay/VGG16_model3.keras")

Epoch 1/50
163/163 [==============================] - 153s 734ms/step - loss: 0.4316 - accuracy: 0.7968 - val_loss: 0.6619 - val_accuracy: 0.6875
Epoch 2/50
163/163 [==============================] - 113s 675ms/step - loss: 0.2856 - accuracy: 0.8848 - val_loss: 0.9237 - val_accuracy: 0.5625
Epoch 3/50
163/163 [==============================] - 112s 676ms/step - loss: 0.2715 - accuracy: 0.8884 - val_loss: 0.7991 - val_accuracy: 0.7500
Epoch 4/50
163/163 [==============================] - 111s 674ms/step - loss: 0.2306 - accuracy: 0.9116 - val_loss: 0.8481 - val_accuracy: 0.6250
Epoch 5/50
163/163 [==============================] - 112s 676ms/step - loss: 0.2376 - accuracy: 0.9097 - val_loss: 0.4571 - val_accuracy: 0.7500
Epoch 6/50
163/163 [==============================] - 112s 675ms/step - loss: 0.2184 - accuracy: 0.9164 - val_loss: 0.8919 - val_accuracy: 0.7500
Epoch 7/50
163/163 [==============================] - 111s 673ms/step - loss: 0.2257 - accuracy: 0.9145 - val_loss: 0.5249 -

In [14]:
model.evaluate(test_dataset)

20/20 [==============================] - 15s 662ms/step - loss: 0.3649 - accuracy: 0.8109


[0.3648855686187744, 0.8108974099159241]

We can see the accuracy on test dataset hasn't increased after running for full 50 epochs, so we'll take our previous model as final model with 81% accuracy.